## PySAL Change Log Statistics: Table Generation

This notebook generates the summary statistics for use in the 6-month releases of PySAL, which is now a meta package. 

It assumes the subpackages have been git cloned in a directory below the location of this notebook. It also requires network connectivity for some of the reporting.

Run this notebook after `100-gitcount.ipynb`


In [ ]:
from __future__ import print_function
import os
import json
import re
import sys
import pandas
import subprocess
from subprocess import check_output

#import yaml
from datetime import datetime, timedelta, time

from dateutil.parser import parse
import pytz

utc=pytz.UTC

try:
    from urllib import urlopen
except:
    from urllib.request import urlopen

from release_info import release_date, start_date, PYSALVER

since = datetime.combine(start_date, time(0,0))


In [ ]:
release_date

In [ ]:
CWD = os.path.abspath(os.path.curdir)

In [ ]:
CWD

In [ ]:
with open('frozen.txt', 'r') as package_list:
    packages = package_list.readlines()
    packages = dict([package.strip().split(">=") for package in packages])

In [ ]:
packages

import pysal
packages['pysal'] = pysal.__version__

In [ ]:
import pickle

In [ ]:
issues_closed = pickle.load(open("issues_closed.p", 'rb'))
pulls_closed = pickle.load(open('pulls_closed.p', 'rb'))

In [ ]:
type(issues_closed)

In [ ]:
issues_closed.keys()

In [ ]:
from release_info import get_pypi_info, get_github_info, clone_masters

In [ ]:
#github_releases = get_github_info()

github_releases = pickle.load(open("releases.p", 'rb'))


In [ ]:
from datetime import datetime

In [ ]:
#pysal_date = datetime.strptime('2021-07-31T12:00:00Z', '%Y-%m-%dT%H:%M:%SZ')
pysal_date = release_date
#ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


In [ ]:
pysal_rel = {'version': f'v{PYSALVER}',
            'release_date': release_date}
github_releases['pysal'] = pysal_rel

In [ ]:
github_releases

In [ ]:
from datetime import datetime
datetime.fromtimestamp(0)
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


final_pulls = {}
final_issues = {}
for package in packages:
    filtered_issues = []
    filtered_pulls = []
    released = github_releases[package]['release_date']
    package_pulls = pulls_closed[package]
    package_issues = issues_closed[package]
    for issue in package_issues:
        #print(issue['number'], issue['title'], issue['closed_at'])
        closed = datetime.strptime(issue['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_issues.append(issue)
    final_issues[package] = filtered_issues
    for pull in package_pulls:
        #print(pull['number'], pull['title'], pull['closed_at'])
        closed = datetime.strptime(pull['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_pulls.append(pull)
    final_pulls[package] = filtered_pulls
    print(package, released, len(package_issues), len(filtered_issues), len(package_pulls),
         len(filtered_pulls))

In [ ]:
issue_details = final_issues
pull_details = final_pulls

In [ ]:
packages

In [ ]:
github_releases['pysal']['release_date'] = release_date

In [ ]:
released

In [ ]:
packages.keys()

In [ ]:
spvcm = packages['spvcm']

In [ ]:
## skip packages not released since last meta release

# handle meta
mrd = github_releases['pysal']['release_date']
github_releases['pysal']['release_date'] =  datetime.combine(mrd, time(0,0))


In [ ]:
for package in github_releases:
    if github_releases[package]['release_date']>since:
        print("new: ",package)
    else:
        print('old:', package)

In [ ]:
github_releases[package]['release_date']

In [ ]:
since

In [ ]:
github_releases

In [ ]:
since_date = '--since="{start}"'.format(start=start_date.strftime("%Y-%m-%d"))
since_date
                                        

In [ ]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
tag_dates = {}
ncommits_total = 0
for subpackage in packages:
    released = github_releases[subpackage]['release_date']
    tag_date = released.strftime("%Y-%m-%d")
    tag_dates[subpackage] = tag_date
    print(tag_date)
    #tag_date = tag_dates[subpackage]
    ncommits = 0
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

In [ ]:
activity

In [ ]:
subpackage

In [ ]:
CWD

In [ ]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
for subpackage in packages:
    ncommits = 0
    tag_date = tag_dates[subpackage]
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        print(ncommits)
        ncommits_total = len(check_output(cmd).splitlines())
        print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

In [ ]:
identities = {'Levi John Wolf': ('ljwolf', 'Levi John Wolf'),
              'Serge Rey': ('Serge Rey', 'Sergio Rey', 'sjsrey', 'serge'),
              'Wei Kang': ('Wei Kang', 'weikang9009'),
              'Dani Arribas-Bel': ('Dani Arribas-Bel', 'darribas'),
              'Antti Härkönen': ( 'antth', 'Antti Härkönen', 'Antti Härkönen', 'Antth'  ),
              'Juan C Duque': ('Juan C Duque', "Juan Duque"),
              'Renan Xavier Cortes': ('Renan Xavier Cortes', 'renanxcortes', 'Renan Xavier Cortes'   ),
              'Taylor Oshan': ('Tayloroshan', 'Taylor Oshan', 'TaylorOshan'),
              'Tom Gertin': ('@Tomgertin', 'Tom Gertin', '@tomgertin')
}

def regularize_identity(string):
    string = string.decode()
    for name, aliases in identities.items():
        for alias in aliases:
            if alias in string:
                string = string.replace(alias, name)
    if len(string.split(' '))>1:
        string = string.title()
    return string.lstrip('* ')

In [ ]:
author_cmd = ['git', 'log', '--format=* %aN', since_date]

In [ ]:
author_cmd.append('blank')

In [ ]:
author_cmd

In [ ]:
from collections import Counter

In [ ]:
tag_dates

In [ ]:
authors_global = set()
authors = {}
global_counter = Counter()
counters = dict()
cmd = ['git', 'log', '--oneline', since_date]
total_commits = 0
activity = {}
for subpackage in packages:
    ncommits = 0
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        ncommits = len(check_output(cmd).splitlines())
        print(cmd)
        tag_date = tag_dates[subpackage]
        tag_date = (datetime.strptime(tag_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        author_cmd[-1] = '--until="{tag_date}"'.format(tag_date=tag_date)
        #cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        print(subpackage, author_cmd)


        all_authors = check_output(author_cmd).splitlines()
        counter = Counter([regularize_identity(author) for author in all_authors])
        global_counter += counter
        counters.update({subpackage: counter})
        unique_authors = sorted(set(all_authors))
        authors[subpackage] =  unique_authors
        authors_global.update(unique_authors)
    total_commits += ncommits
    activity[subpackage] = ncommits

In [ ]:
author_cmd

In [ ]:
subpackage

In [ ]:
authors_global

In [ ]:
activity

In [ ]:
def get_tag(title, level="##", as_string=True):
    words = title.split()
    tag = "-".join([word.lower() for word in words])
    heading = level+" "+title
    line = "\n\n<a name=\"{}\"></a>".format(tag)
    lines = [line]
    lines.append(heading)
    if as_string:
        return "\n".join(lines)
    else:
        return lines

In [ ]:
subs = issue_details.keys()
table = []
txt = []
lines = get_tag("Changes by Package", as_string=False)

for sub in subs:
    total= issue_details[sub]
    pr = pull_details[sub]
    
    row = [sub, activity[sub], len(total), len(pr)]
    table.append(row)
    #line = "\n<a name=\"{sub}\"></a>".format(sub=sub)
    #lines.append(line)
    #line = "### {sub}".format(sub=sub)
    #lines.append(line)
    lines.extend(get_tag(sub.lower(), "###", as_string=False))
    for issue in total:
        url = issue['html_url']
        title = issue['title']
        number = issue['number']
        line = "* [#{number}:]({url}) {title} ".format(title=title,
                                                     number=number,
                                                     url=url)
        lines.append(line)



In [ ]:
sub

In [ ]:
os.chdir(CWD)

import pandas

In [ ]:
df = pandas.DataFrame(table, columns=['package', 'commits', 'total issues', 'pulls'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
type(counters)

In [ ]:
df.sort_values(['commits','pulls'], ascending=False)\
  .to_html('./commit_table.html', index=None)

In [ ]:
df.sum()

In [ ]:
contributor_table = pandas.DataFrame.from_dict(counters).fillna(0).astype(int).T

In [ ]:
contributor_table.to_html('./contributor_table.html')

In [ ]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits')

In [ ]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits').to_html('./commits_by_person.html')

In [ ]:
totals

In [ ]:
n_commits = df.commits.sum()
n_issues = df['total issues'].sum()
n_pulls = df.pulls.sum()

In [ ]:
n_commits

In [ ]:
#Overall, there were 719 commits that closed 240 issues, together with 105 pull requests across 12 packages since our last release on 2017-11-03.
#('{0} Here is a really long '
#           'sentence with {1}').format(3, 5))
line = ('Overall, there were {n_commits} commits that closed {n_issues} issues'  
    ' since our last release' 
        ' on {since_date}.\n'.format(n_commits=n_commits, n_issues=n_issues,
        since_date = start_date))

In [ ]:
line

## append html files to end of changes.md with tags for toc

In [ ]:
with open('changes.md', 'w') as fout:
    fout.write(line)
    fout.write("\n".join(lines))
    fout.write(get_tag("Contributors"))
    fout.write("\n\nMany thanks to all of the following individuals who contributed to this release:\n\n")
    
    
    
    totals = contributor_table.sum(axis=0).T
    contributors = totals.index.values
    contributors.sort()
    contributors = contributors.tolist() 
    contributors = [ f'\n - {contributor}' for contributor in contributors]
    fout.write("".join(contributors))
    


In [ ]:
df.head()

In [ ]:
df.head(17)

In [ ]:
n_pulls

In [ ]:
"""
Update ../pyproject.toml for minimum pysal package pinning
"""
# get version numbers from frozen.txt
with open('frozen.txt', 'r') as frozen:
    packages = [line.rstrip() for line in frozen.readlines()]

# search pyproject.toml for lines containing package
with open('../pyproject.toml', 'r') as project:
    lines = [line.rstrip() for line in project.readlines()]

# split line ->"    package",  ">=",  "version",
# replace version and rebuild line to update
for package in packages:
    name, version = package.split(">=")
    i, match = [(i, line) for i, line in enumerate(lines) if name in line][0]
    old_name, old_version = match.split(">=")
    new_line = ">=".join([old_name, version+'",'])
    lines[i] = new_line

# write out new pyproject.toml file
with open("../pyproject.toml", 'w') as output:
    output.write("\n".join(lines))
